In [108]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()

from google.colab import drive
drive.mount('/content/drive')



Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [109]:
import torch.optim as optim

def accuracy(out, labels):
  _,pred = torch.max(out, dim=1)
  return torch.sum(pred==labels).item()


num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1)
t=torch.ones([1], dtype=torch.float64)
t[0]=0.4853
t= t.to('cuda')
unb_criterion = torch.nn.BCEWithLogitsLoss(weight =t)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [110]:
from torchvision.transforms.transforms import Normalize
import numpy as np
import pandas as pd
import imageio 
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import os
import albumentations as A
import cv2
!pip install albumentations==0.4.6
from albumentations.pytorch.transforms import ToTensorV2


preprocess_train = A.Compose([
    A.Normalize(max_pixel_value=1.0),
    A.RandomCropNearBBox(0.15),
    A.HorizontalFlip(),
    ToTensorV2(),


])
preprocess_val = A.Compose([
    A.Normalize(max_pixel_value=1.0),
    A.RandomCropNearBBox(0),
    ToTensorV2(),


])



class MyDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform,train,dataset="tiff_wb"):
        split = "train" if train else "val"
        self.brix= pd.read_csv(csv_file,sep=",")
        self.root_dir = os.path.join(root_dir, dataset, split)
        self.transform = transform

        self.brix = self.brix[self.brix["split"] == split]
        self.brix = self.brix.reset_index()
        #self.img_labels = csv[["file_id", "brix", "label"]]

    def __len__(self):
        return len(self.brix)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
  
        l=self.brix.iloc[idx,2].split(".")
        name=l[0]+".tiff"
        img_name = os.path.join(self.root_dir, name)

        image = imageio.imread(img_name)
        # print("img", np.max(image))
        image = np.asarray(image, dtype=np.float32)  # convert imageio to ndarray
        image = image / (2**16-1)  # normalize between 0 and 1

        # Cropping box
        box_path = os.path.join('/content/drive/MyDrive/canopies-utilities', 'bboxes', f'{l[0]}.txt')
        bbox = np.loadtxt(box_path, delimiter=" ", dtype=np.float32)
        bbox = bbox[1:]
        # Convert to pascal format xmin ymin xmax ymax
        x_min = int((bbox[0] - bbox[2] / 2.0) * 1500)
        x_max = int((bbox[0] + bbox[2] / 2.0) * 1500)
        y_min = int((bbox[1] - bbox[3] / 2.0) * 2000)
        y_max = int((bbox[1] + bbox[3] / 2.0) * 2000)
        
        if self.transform is not None:
          image = self.transform(image=image, cropping_bbox=[x_min, y_min, x_max, y_max])["image"]

        else:
        #image = torch.as_tensor(image['image'])  # convert to tensor
          image = torch.as_tensor(image)
          image = image.permute((2, 0, 1))  # permute the image from HWC to CHW format
        image=transforms.Resize((512,512))(image)
        # print(image.dtype)
        # print(image.shape)
        label=np.float32(self.brix.iloc[idx,4])
        sample = {'image': image, 'label': label}
        # print("img", np.max(sample['image']))

        return sample

#brix_dataset = MyDataset(csv_file=open('/content/drive/MyDrive/datasets/brix_labels.csv', 'r'),
                                     #root_dir='drive/MyDrive/datasets',transform=None,train=True)
train_dataset = MyDataset(csv_file=open('/content/drive/MyDrive/datasets/brix_labels.csv', 'r'),
                                     root_dir='drive/MyDrive/datasets',transform=preprocess_train,train=True)
val_dataset = MyDataset(csv_file=open('/content/drive/MyDrive/datasets/brix_labels.csv', 'r'),
                                     root_dir='drive/MyDrive/datasets',transform=preprocess_val,train=False)
                           
sample = train_dataset[2]
input_image=sample['image']
print(sample['label'])
input_tensor=input_image
input_batch = input_tensor.unsqueeze(0)  # create a mini-batch as expected by the model

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
    print(output)
    print(input_batch.shape)
probabilities = torch.sigmoid(output[0])
print(input_batch)
print("Prob:", probabilities)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
1.0
tensor([[-0.1233]], device='cuda:0')
torch.Size([1, 3, 512, 512])
tensor([[[[-0.6186, -0.9823, -1.3343,  ...,  1.9762,  1.6321,  1.3270],
          [ 0.7600, -0.7441, -1.3930,  ...,  2.2489,  2.1914,  2.0151],
          [-0.8183, -1.1823, -1.1610,  ...,  2.2489,  2.2489,  2.2489],
          ...,
          [-1.9115, -1.9128, -1.9066,  ..., -0.8254, -0.8816, -1.5127],
          [-1.8921, -1.9097, -1.8817,  ..., -0.7627, -1.0823, -1.5046],
          [-1.8995, -1.9004, -1.9158,  ..., -1.2906, -1.4645, -1.4885]],

         [[-1.1519, -1.1523, -1.4613,  ...,  1.4482,  1.0983,  0.7779],
          [-0.1985, -0.8213, -1.4736,  ...,  2.0017,  1.7072,  1.4634],
          [-1.0966, -1.4140, -1.4135,  ...,  2.3501,  2.2626,  2.1148],
          ...,
          [-1.8068, -1.8126, -1.8115,  ..., -1.3388, -1.3713, -1.5742],
          [-1.8113, -1.8177, -1.8260,  ..., -1.3391, -1.3642, -1.5949],
       

In [111]:
from torch.utils.data import DataLoader
from sklearn.utils import resample

#training_data,test_data=torch.utils.data.random_split(brix_dataset,[120,30])
#train_dataloader = DataLoader(training_data, batch_size=6, shuffle=True)
#test_dataloader = DataLoader(test_data, batch_size=6, shuffle=True)


train_dataloader = DataLoader(train_dataset, batch_size=6, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1)

In [112]:
import sklearn.metrics

def evaluate(model,dataloader,criterion,device):
  model.eval()
  with torch.no_grad():
    val_loss=0.0
    preds_all=[]
    labels_all=[]
    i=0
    for data in dataloader:
      data_= data['image']
      target_=data['label']
      #target_=torch.tensor(target_).unsqueeze(0)
      if torch.cuda.is_available():
        data_, target_ = data_.to(device), target_.to(device)
    
      outputs=model(data_)
      outputs=outputs.reshape(data_.size(0))
      pred=1 if torch.sigmoid(outputs).item()>0.5 else 0
      preds_all.append(pred)
      labels_all.append(int(target_.item()))
      loss=criterion(outputs,target_)
      val_loss+=(loss.item()-val_loss/(i+1)) #calcolo la media volta per volta
      i+=1
    f1_score=sklearn.metrics.f1_score(labels_all,preds_all)
    print(f'Validation loss: {val_loss:.4f}')
    print(f'F1 score: {f1_score:.4f}')
    #print(f'Predictions: {preds_all}')
    #print(f'Labels: {labels_all}')
  model.train()
  return val_loss

        


In [113]:
n_epochs = 20
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = 0
dataset_size=len(train_dataset)
preds_all=[]
labels_all=[]

model.train()

for epoch in range(1, n_epochs+1):
  running_loss = 0.0
  correct = 0
  total=0
  print(f'Epoch {epoch}\n')
  
  for data in train_dataloader:
    data_= data['image']
    target_=data['label']
    #data_ = data_.unsqueeze(0) 
    #target_=torch.tensor(target_).unsqueeze(0)
    if torch.cuda.is_available():
      data_, target_ = data_.to('cuda'), target_.to('cuda')
    optimizer.zero_grad()
    
    target_=target_.reshape([6])
    #print(data_.shape)     
    outputs = model(data_)
    #print(outputs,target_)
    outputs=outputs.reshape(data_.size(0))
    #pred=1 if torch.sigmoid(outputs).item() >0.5 else 0
    #preds_all.append(pred)
    #labels_all.append(int(target_.item()))
    loss = unb_criterion(outputs, target_)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
  print(f'\ntrain-loss: {train_loss/dataset_size:.4f}')
  #f1_score=sklearn.metrics.f1_score(labels_all,preds_all)
  #print(f'F1 score: {f1_score:.4f}')
  train_loss = 0
  # Evaluate model after each epoch
  evaluate(model,val_dataloader,criterion,'cuda')

Epoch 1


train-loss: 0.0546
Validation loss: 3.8662
F1 score: 0.6667
Epoch 2


train-loss: 0.0510
Validation loss: 4.1638
F1 score: 0.6667
Epoch 3


train-loss: 0.0505
Validation loss: 4.3069
F1 score: 0.6667
Epoch 4


train-loss: 0.0488
Validation loss: 4.4076
F1 score: 0.6667
Epoch 5


train-loss: 0.0489
Validation loss: 4.3932
F1 score: 0.6667
Epoch 6


train-loss: 0.0492
Validation loss: 4.3620
F1 score: 0.6667
Epoch 7


train-loss: 0.0487
Validation loss: 4.4657
F1 score: 0.6667
Epoch 8


train-loss: 0.0489
Validation loss: 4.5128
F1 score: 0.6667
Epoch 9


train-loss: 0.0490
Validation loss: 4.6705
F1 score: 0.6667
Epoch 10


train-loss: 0.0477
Validation loss: 4.6575
F1 score: 0.6667
Epoch 11


train-loss: 0.0476
Validation loss: 4.7132
F1 score: 0.6667
Epoch 12


train-loss: 0.0472
Validation loss: 4.5807
F1 score: 0.6667
Epoch 13


train-loss: 0.0491
Validation loss: 4.6327
F1 score: 0.6667
Epoch 14


train-loss: 0.0468
Validation loss: 4.7160
F1 score: 0.6667
Epoch 15


trai

In [105]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
model.eval()
with torch.no_grad():
    for data in val_dataloader:
        images= data['image']
        labels=data['label']
        images, labels = images.cuda(), labels.cuda()
        #images, labels = images.to('cuda'), labels.to('cuda')
        
        # calculate outputs by running images through the network
        outputs = model(images)
        probabilities = torch.sigmoid(outputs)
        #print(outputs.shape,probabilities.shape,labels.shape)
        # the class with the highest energy is what we choose as prediction
        #_, predicted = torch.max(outputs.data, 1)
        pred=torch.zeros([probabilities.shape[0]])
        print(probabilities)
        pred=pred.cuda()
        for i in range(probabilities.shape[0]):
          if(probabilities[i]>0.5):
            pred[i]=1

        #print(probabilities,labels)
        #print(labels.size(0)))
        #print(pred)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
        print(correct)

print(f'Accuracy of the network on the 10 test images: {100 * correct // total} %')

tensor([[0.5557]], device='cuda:0')
1
tensor([[0.5915]], device='cuda:0')
1
tensor([[0.6439]], device='cuda:0')
2
tensor([[0.6543]], device='cuda:0')
2
tensor([[0.6789]], device='cuda:0')
3
tensor([[0.7525]], device='cuda:0')
3
tensor([[0.7670]], device='cuda:0')
3
tensor([[0.8040]], device='cuda:0')
4
tensor([[0.8638]], device='cuda:0')
4
tensor([[0.7967]], device='cuda:0')
5
Accuracy of the network on the 10 test images: 50 %
